# Introducción al uso de interfaces de datos (API) (III)

## Ejercicio: Consulta multiples series a OpenMeteo

En este ejercicio vamos a trabajar en realizar consultas a la API de OpenMeteo en Icod de Los Vinos para obtener 4 series diferentes: temperatura a 2 metros, humedad relativa a 2 metros, probabilidad de precipitación y velocidad del viento a 10 metros.

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json

# Coordenadas de Icod de Los Vinos
latitud = 28.367
longitud = -16.721

# URL de la API de OpenMeteo para obtener las 4 series de datos
url = f"https://api.open-meteo.com/v1/forecast?latitude={latitud}&longitude={longitud}&hourly=temperature_2m,relative_humidity_2m,precipitation_probability,wind_speed_10m"

# Realizar la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta en JSON
    data = response.json()

    print("Consulta ejecutada correctamente.")
    # Guardamos datos en formato JSON
    with open('data/m2_e3_datos_meteo_multiseries.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)
else:
    print(f"Error en la solicitud: {response.status_code}")


### Leer datos en formato JSON
Después de guardar la respuesta, leemos el archivo JSON usando json.load() para cargar los datos en una variable llamada data.

In [ ]:
with open('data/m2_e3_datos_meteo_multiseries.json', 'r') as json_file:
    data = json.load(json_file)

data['hourly'].keys()

### Creación del dataframe
Usamos los datos leídos del archivo JSON para crear un DataFrame con 4 columnas: Fecha, Temperatura (°C), Humedad Relativa (%), Probabilidad Precipitación (%) y Velocidad Viento (km/h).

In [ ]:
# Crear un DataFrame con las 4 series de datos
df = pd.DataFrame({
    'Fecha': pd.to_datetime(data['hourly']['time']),  # Convertir las horas a datetime
    'Temperatura (°C)': data['hourly']['temperature_2m'],
    'Humedad Relativa (%)': data['hourly']['relative_humidity_2m'],
    'Probabilidad Precipitación (%)': data['hourly']['precipitation_probability'],
    'Velocidad Viento (km/h)': data['hourly']['wind_speed_10m']
})

# Mostrar las primeras filas del DataFrame
print(df.head())

In [ ]:
# Guardar el DataFrame en un archivo CSV
df.to_csv('data/m2_e3_datos_meteo_multiserie.csv', index=False)
print("Datos guardados en 'm2_1_3_datos_meteo_multiserie.csv'.")

### Visualización combinada series temporales
* Graficamos las 4 series en un solo gráfico, cada una con un color diferente para facilitar la comparación.
* Las etiquetas del eje X se rotan 45 grados para mejorar la legibilidad.
* Se añade una leyenda para identificar cada serie, junto con una cuadrícula para mejorar la visualización.

In [ ]:
# Visualización de las 4 series en un gráfico
plt.figure(figsize=(12, 8))

# Graficar las 4 series
plt.plot(df['Fecha'], df['Temperatura (°C)'], color='blue', label='Temperatura (°C)')
plt.plot(df['Fecha'], df['Humedad Relativa (%)'], color='green', label='Humedad Relativa (%)')
plt.plot(df['Fecha'], df['Probabilidad Precipitación (%)'], color='purple', label='Probabilidad Precipitación (%)')
plt.plot(df['Fecha'], df['Velocidad Viento (km/h)'], color='orange', label='Velocidad Viento (km/h)')

# Añadir etiquetas y título
plt.title('Series Temporales de Variables Meteorológicas en Icod de Los Vinos', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valores', fontsize=12)

# Rotar las etiquetas del eje X
plt.xticks(rotation=45)

# Mostrar la leyenda
plt.legend()

# Mostrar la cuadrícula
plt.grid(True)

# Ajustar los márgenes
plt.tight_layout()

# Mostrar el gráfico
plt.show()

Podemos ver las 4 series en simultáneo en una misma gráfica. Sin emabrgo, hay algo que no cuadra en esta gráfica ???

#### Gráfico con múltiples índices
Para visualizar múltiples series con unidades diferentes en un mismo gráfico, podemos utilizar ejes Y independientes para cada serie. En Matplotlib, esto se logra utilizando la función twinx(), que crea un segundo eje Y que comparte el mismo eje X.

In [ ]:
# Crear el gráfico principal con el primer eje Y (temperatura)
fig, ax1 = plt.subplots(figsize=(12, 6))

# Graficar la temperatura en el eje Y1
ax1.set_xlabel('Fecha')
ax1.set_ylabel('Temperatura (°C)', color='blue')
ax1.plot(df['Fecha'], df['Temperatura (°C)'], color='blue', label='Temperatura (°C)')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True)

# Crear un segundo eje Y que comparta el eje X (humedad relativa)
ax2 = ax1.twinx()  
ax2.set_ylabel('Humedad Relativa (%)', color='green')
ax2.plot(df['Fecha'], df['Humedad Relativa (%)'], color='green', label='Humedad Relativa (%)')
ax2.tick_params(axis='y', labelcolor='green')

# Crear un tercer eje Y para la probabilidad de precipitación
ax3 = ax1.twinx()  
ax3.spines['right'].set_position(('outward', 60))  # Mover el tercer eje un poco a la derecha
ax3.set_ylabel('Probabilidad Precipitación (%)', color='purple')
ax3.plot(df['Fecha'], df['Probabilidad Precipitación (%)'], color='purple', label='Probabilidad Precipitación (%)')
ax3.tick_params(axis='y', labelcolor='purple')

# Crear un cuarto eje Y para la velocidad del viento
ax4 = ax1.twinx()  
ax4.spines['right'].set_position(('outward', 120))  # Mover el cuarto eje más a la derecha
ax4.set_ylabel('Velocidad Viento (km/h)', color='orange')
ax4.plot(df['Fecha'], df['Velocidad Viento (km/h)'], color='orange', label='Velocidad Viento (km/h)')
ax4.tick_params(axis='y', labelcolor='orange')

# Rotar las etiquetas del eje X para mayor legibilidad
plt.xticks(rotation=45)

# Añadir el título del gráfico
plt.title('Variables Meteorológicas en Icod de Los Vinos', fontsize=16)

# Ajustar los márgenes
fig.tight_layout()

# Mostrar el gráfico
plt.show()


### Multiples gráficos
Para resolver el problema de visualización y mostrar cada serie meteorológica en su propio gráfico dentro de una sola figura, podemos utilizar **subplots** en Matplotlib. Esto permitirá crear un gráfico separado para cada variable en un diseño de múltiple gráfico dentro de una sola figura.

In [ ]:
# Crear un multiplot con 4 gráficos en una sola figura
fig, axs = plt.subplots(4, 1, figsize=(10, 12), sharex=True)

# Gráfico de Temperatura
axs[0].plot(df['Fecha'], df['Temperatura (°C)'], color='blue')
axs[0].set_title('Temperatura (°C)')
axs[0].set_ylabel('°C')
axs[0].grid(True)

# Gráfico de Humedad Relativa
axs[1].plot(df['Fecha'], df['Humedad Relativa (%)'], color='green')
axs[1].set_title('Humedad Relativa (%)')
axs[1].set_ylabel('%')
axs[1].grid(True)

# Gráfico de Probabilidad de Precipitación
axs[2].plot(df['Fecha'], df['Probabilidad Precipitación (%)'], color='purple')
axs[2].set_title('Probabilidad de Precipitación (%)')
axs[2].set_ylabel('%')
axs[2].grid(True)

# Gráfico de Velocidad del Viento
axs[3].plot(df['Fecha'], df['Velocidad Viento (km/h)'], color='orange')
axs[3].set_title('Velocidad del Viento (km/h)')
axs[3].set_ylabel('km/h')
axs[3].set_xlabel('Fecha')
axs[3].grid(True)

# Ajustar los márgenes y mostrar la figura
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Histograma de múltiples series temporales
Creamos un multiplot de histogramas de las cuatro series (temperatura, humedad relativa, probabilidad de precipitación y velocidad del viento) en una sola figura, utilizando subplots en Matplotlib:

In [ ]:
# Crear un multiplot de histogramas
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

# Histograma de Temperatura
axs[0, 0].hist(df['Temperatura (°C)'], bins=20, color='blue', edgecolor='black')
axs[0, 0].set_title('Temperatura (°C)')
axs[0, 0].set_xlabel('°C')
axs[0, 0].set_ylabel('Frecuencia')

# Histograma de Humedad Relativa
axs[0, 1].hist(df['Humedad Relativa (%)'], bins=20, color='green', edgecolor='black')
axs[0, 1].set_title('Humedad Relativa (%)')
axs[0, 1].set_xlabel('%')
axs[0, 1].set_ylabel('Frecuencia')

# Histograma de Probabilidad de Precipitación
axs[1, 0].hist(df['Probabilidad Precipitación (%)'], bins=20, color='purple', edgecolor='black')
axs[1, 0].set_title('Probabilidad de Precipitación (%)')
axs[1, 0].set_xlabel('%')
axs[1, 0].set_ylabel('Frecuencia')

# Histograma de Velocidad del Viento
axs[1, 1].hist(df['Velocidad Viento (km/h)'], bins=20, color='orange', edgecolor='black')
axs[1, 1].set_title('Velocidad del Viento (km/h)')
axs[1, 1].set_xlabel('km/h')
axs[1, 1].set_ylabel('Frecuencia')

# Ajustar los márgenes entre gráficos
plt.tight_layout()

# Mostrar el gráfico
plt.show()